# 运行速度的一些测试和总结

## 全局设置

In [ ]:
from functools import wraps
from random import shuffle
from time import time
from typing import Callable, Union

temp = [i for i in range(1_000_000)]
shuffle(temp)
FROZEN_LIST: tuple = tuple(temp)
del temp

## 计时器

In [ ]:
def timer(arg: Union[int, Callable]):
    if isinstance(arg, int):
        def decorator(func: Callable):
            @wraps(func)
            def wrapper(*args, **kwargs):
                result = []
                before = time()
                for _ in range(arg):
                    result.append(func(*args, **kwargs))
                after = time()
                print(f"runtime = {after - before} seconds")
                if any(i is not None for i in result):
                    return result
            return wrapper
        return decorator
    else:
        func = arg
        @wraps(func)
        def wrapper(*args, **kwargs):
            before = time()
            result = func(*args, **kwargs)
            after = time()
            print(f"runtime = {after - before} seconds")
            return result
        return wrapper

## 引用和索引的比较

In [ ]:
list_ = list(FROZEN_LIST)
index: int
value: int

@timer(1_000_000)
def foo():
    value

@timer(1_000_000)
def bar():
    list_[index]

for i in range(6):
    index = 10 ** (i) - 1
    value = list_[index]
    print(f"index = {index}")  # f"{index = }" in Python 3.8 and later
    print("foo: ", end="")
    foo()
    print("bar: ", end="")
    bar()
    print("\n", end="")  # print("") is less explicit

del foo
del bar

结论:

- 引用 (全局 `__dict` 属性中取键值) 要快于列表中取索引.

## 元组转列表

In [ ]:
for i in range(4):
    looptimes: int = 10 ** i

    @timer(looptimes)
    def foo():
        list_ = []
        list_[:] = FROZEN_LIST

    @timer(looptimes)
    def foo_():
        list_ = []
        list_[:] = FROZEN_LIST[:]

    @timer(looptimes)
    def bar():
        list_ = list(FROZEN_LIST)

    @timer(looptimes)
    def baz():
        list_ = [*FROZEN_LIST]

    print(f"looptimes = {looptimes}")
    print("foo: ", end="")
    foo()
    print("foo_: ", end="")
    foo_()
    print("bar: ", end="")
    bar()
    print("baz: ", end="")
    baz()
    print("\n", end="")

del foo
del foo_
del bar
del baz

结论:

- 单次运行时, 内置函数 `list` (实际为内置类) 平均运行速度较快, 但存在解包快于 `list` 的情况.
- 循环运行时, 解包明显快于其他方法, 原因不明.

鉴于元组转列表一般为单次操作, 且解包的**可读性**不如 `list` 函数, 推荐使用后者.